### Reload .py files whenever there are changes

In [1]:
%load_ext autoreload
%autoreload 2

### Local imports from this project

In [2]:
from mdf import *
from mdf.ops import *
from mdf.data import *

### External imports from dependencies

In [3]:
from pytorch3d.ops.laplacian_matrices import norm_laplacian, cot_laplacian, laplacian
from pytorch3d.ops import norm_laplacian, cot_laplacian, laplacian
from pyvista.plotting.plotter import Plotter
import torchvision.transforms.v2 as T
import torch.distributions as distrib
import plotly.graph_objects as go
import plotly.express as px
from torch import Tensor
from typing import cast
import torch_geometric
import torch.nn as nn
import pyvista as pv
import scipy.linalg
import torch.linalg
import numpy as np
import pytorch3d
import einops
import torch

### Environment setup

In [4]:
pv.set_jupyter_backend('trame')
torch.manual_seed(42)
np.random.seed(42)

### Data Loading

In [5]:
# Download standard shapes
data = DataManager(data_dir=data_dir, cache_dir=cache_dir)

# Choose a Manifold and a Signal
signal = cast(t.Any, data.weather[5500]['data'])
manifold = cast(t.Any, data.objects['stanford-bunny'].vista)

# Sample a couple of points
k_evecs = 100
device = torch.device('cuda')
manifold_sampler = ManifoldSampler(torch.tensor(manifold.points), torch.tensor(manifold.faces.reshape((-1, 4))), k_evecs=k_evecs, device=device)

/home/invokariman/.cache/pypoetry/virtualenvs/mdf-B-9uJKhD-py3.11/lib/python3.11/site-packages/pytorch3d/io/obj_io.py:546: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


### How to apply a texture to a 3d mesh

![https://github.com/pyvista/pyvista-support/issues/168](https://user-images.githubusercontent.com/22067021/83365096-c2069800-a373-11ea-9cf3-418f9e55147c.png)

In [6]:
p = Plotter()
texture = pv.Texture(signal)
manifold.texture_map_to_sphere(inplace=True)
p.add_mesh(manifold, texture=texture)
p.camera.tight()
p.show()

Widget(value='<iframe src="http://localhost:41195/index.html?ui=P_0x790df2f32390_0&reconnect=auto" class="pyvi…

### Compute Positional Embedding using Laplace-Beltrami Operator

In [7]:
# Create 3d mesh
mesh = pv.PolyData(manifold_sampler.verts.numpy(), manifold_sampler.faces.numpy())
evecs = manifold_sampler.embed(torch.arange(manifold_sampler.verts.size(0)))

# Plot multiple images
p = Plotter(shape=(1, 3))

# 1st eigenfunction
p.subplot(0, 0)
p.add_mesh(mesh.copy(), scalars=evecs[:, 1:2].sum(dim=1), clim=[-1, 1], cmap='RdBu_r')
p.camera.tight()

# 2nd eigenfunction
p.subplot(0, 1)
p.add_mesh(mesh.copy(), scalars=evecs[:, 51:52].sum(dim=1), clim=[-1, 1], cmap='RdBu_r')
p.camera.tight()

# 3rd eigenfunction
p.subplot(0, 2)
p.add_mesh(mesh.copy(), scalars=evecs[:, 99:100].sum(dim=1), clim=[-1, 1], cmap='RdBu_r')
p.camera.tight()

p.show()

Widget(value='<iframe src="http://localhost:41195/index.html?ui=P_0x790dc1307e50_1&reconnect=auto" class="pyvi…

In [8]:
pts: PointsOnManifold = manifold_sampler.sample(n=30000, image=torch.tensor(signal))
mesh = pv.PolyData(pts['pos'].numpy())
pe: Tensor = pts['pos_embedding']

# Plot multiple images
p = Plotter(shape=(1, 3))

# 1st eigenfunction
p.subplot(0, 0)
p.add_mesh(mesh.copy(), scalars=pe[:, 1], clim=[-1, 1], cmap='RdBu_r')
p.camera.tight()

# 2nd eigenfunction
p.subplot(0, 1)
p.add_mesh(mesh.copy(), scalars=pe[:, 49], clim=[-1, 1], cmap='RdBu_r')
p.camera.tight()

# 3rd eigenfunction
p.subplot(0, 2)
p.add_mesh(mesh.copy(), scalars=pe[:, 99], clim=[-1, 1], cmap='RdBu_r')
p.camera.tight()

p.show()

Widget(value='<iframe src="http://localhost:41195/index.html?ui=P_0x790dcb6f2ed0_2&reconnect=auto" class="pyvi…

### Function f : M -> Y

In [9]:
p = Plotter(shape=(1, 3))

# Function that uses UV mapping from Pyvista
p.subplot(0, 0)
mesh = pv.PolyData(pts['pos'].numpy())
mesh.texture_map_to_plane(inplace=True)
p.add_mesh(mesh.copy(), texture=texture)
p.camera.tight()

# Function that maps from M to Y
p.subplot(0, 1)
mesh = pv.PolyData(pts['pos'].numpy())
mesh['signal'] = pts['signal']
p.add_mesh(mesh.copy(), scalars='signal', rgb=True)
p.camera.tight()

# Also perform interpolation over the original manifold
p.subplot(0, 2)
mesh = pv.PolyData(manifold_sampler.verts.numpy(), manifold_sampler.faces.numpy())
text = pv.PolyData(pts['pos'].numpy())
text['colors'] = pts['signal']
mesh = mesh.interpolate(text, n_points=3, sharpness=10)
p.add_mesh(text, scalars='colors', rgb=True, render_points_as_spheres=True)
p.add_mesh(mesh, scalars='colors', rgb=True)
p.camera.tight()

p.show()

Widget(value='<iframe src="http://localhost:41195/index.html?ui=P_0x790e85407cd0_3&reconnect=auto" class="pyvi…